In [7]:
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import requests
import re
import random

In [8]:
path2data = 'data/Alsace_166_176_307.csv'
bat_data = pd.read_csv(path2data, sep = ';', encoding='latin1')
# regex accents
bat_data.Texte_Presentation = bat_data.Texte_Presentation.apply(lambda x: re.sub('\x92', " ", x))
num_col = ['Depenses_T2_2017', 'Depenses_T3_2017', 'Depenses_T5_2017', 'Depenses_HT2_Personnel_2017','Effectif_ETPT_2017', 'Actif_Immobilier', 'Actif_Vehicules']
for col in num_col:
    bat_data[col] = bat_data[col].apply(lambda x: np.float(re.sub(' ','', x)))

## Explore dataframe

In [9]:
bat_data.head()

,ID,Type,Fonction,Designation,Latitude,Longitude,Adresse,CodePostal,Ville,Ministere,Programmes,Libelle_Programme,Texte_Presentation,Depenses_T2_2017,Depenses_T3_2017,Depenses_T5_2017,Depenses_HT2_Personnel_2017,Effectif_ETPT_2017,Actif_Immobilier,Actif_Vehicules
0,IB00100028657,Bureau,Tribunal,TRIBUNAL DE GRANDE INSTANCE DE STRASBOURG,48.587333,7.747495,QU FINKMATT,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
1,IB00100036271,Bureau,Tribunal,TRIBUNAL D INSTANCE DE STRASBOURG,48.589145,7.746712,45 R DU FOSSE DES TREIZE,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
2,IB00100028684,Bureau,Tribunal,CONSEIL DE PRUD'HOMMES DE STRASBOURG,48.589834,7.755428,19 AV DE LA PAIX,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
3,IB00100035510,Bureau,Tribunal,TRIBUNAL ADMINISTRATIF STRASBOURG,48.591572,7.756567,31 AV DE LA PAIX,67000,Strasbourg,Services du premier ministre,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
4,IB00100028712,Bureau,Tribunal,TRIBUNAL D INSTANCE DE MOLSHEIM,48.541025,7.494306,3 R DU MARECHAL KELLERMANN,67120,Molsheim,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0


## Build map

In [10]:
def latlon(df):
    col_ = list(df['Ville'].unique())
    res = pd.DataFrame(index=col_)
    for c in col_:
        lat = df.loc[df['Ville']==c, 'Latitude'].iloc[0]
        lon = df.loc[df['Ville']==c, 'Longitude'].iloc[0]
        res.loc[c, 'Latitude'] = lat
        res.loc[c, 'Longitude'] = lon
    return res

loc_cities = latlon(bat_data)
cities = list(loc_cities.index)


def list_min(df, colors_, names_):
    col_ = list(df["Ministere"].unique())
    res = pd.DataFrame(index=col_)
    i = 0
    for c in col_:
        res.loc[c, 'coul'] = colors_[i]
        res.loc[c, 'couleur'] = names_[i]
        i += 1
    return res

def list_prog(df):
    col_ = list(df["Libelle_Programme"].unique())
    col_.append('Tous')
    return col_

def list_fonction(df):
    col_ = list(df["Fonction"].unique())
    col_.append('Tous')
    return col_

In [12]:
colors = ["#FFCE00","#007849", "#E37222","#0375B4","#FFCE00","#0A8A9F"]
names = ['orange', 'green', 'tangerine', 'blue', 'yellow', 'teal']
minis = list_min(bat_data, colors, names)
print(minis['couleur'])
fonction = list_fonction(bat_data)

@interact(tiles=map_styles, cities_ = cities, fonction_ = fonction, zoom=(1, 18))
def create_map(tiles="Stamen Toner", zoom=13, cities_ = cities[0], fonction_= 'Tous'):
    
    try:
        lat = loc_cities.loc[cities_, 'Latitude']
        lon = loc_cities.loc[cities_, 'Longitude']
    except KeyError:
        lat = 48.5734
        lon = 7.7521113
        pass
    
    folium_map = folium.Map(location=[lat, lon], zoom_start=zoom, tiles=tiles, width = '100%')
    
    if fonction_ == 'Tous':
        for idx, row in bat_data.iterrows():

            # generate the popup message that is shown on click.
            num_layus = "Pour répondre à ses missions, en 2017: \
            <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
            <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
            <li>Il a investi {invest}M€ en 2017</li> </ul>  \
            Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
            et véhicules pour une valeur de {vehicule}M€."
    
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
            <b>Fonction:</b> {}<br> <br><b>Pogramme budgétaire:</b> {} <br><br> <b>Données budgétaires et patrimoniales:</b> <br>{}"
    
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  #row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'],
                                  num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                                   invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))

            # radius of circles
            radius = 10
            color = minis.loc[row["Ministere"], 'coul']

            # add marker to the map
            folium.CircleMarker(location=(row["Latitude"],
                                          row["Longitude"]),
                                radius=radius,
                                color=color,
                                popup=popup_text,
                                fill=True).add_to(folium_map)
            
            
    else:
        bb = bat_data.loc[bat_data["Fonction"]==fonction_,]
        for idx, row in bb.iterrows():

            # generate the popup message that is shown on click.
            num_layus = "Pour répondre à ses missions, en 2017: \
            <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
            <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
            <li>Il a investi {invest}M€ en 2017</li> </ul>  \
            Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
            et véhicules pour une valeur de {vehicule}M€."
    
            popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
            <b>Fonction:</b> {}<br> <br><b>Pogramme budgétaire:</b> {} <br><br> <b>Données budgétaires et patrimoniales:</b> <br>{}"
    
            popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                                  row["Ministere"],
                                  row["Fonction"],
                                  #row['Effectif_ETPT_2017'],
                                  row['Texte_Presentation'],
                                  num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                                   invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))

            # radius of circles
            radius = 10
            color = minis.loc[row["Ministere"], 'coul']

            # add marker to the map
            folium.CircleMarker(location=(row["Latitude"],
                                          row["Longitude"]),
                                radius=radius,
                                color=color,
                                popup=popup_text,
                                fill=True).add_to(folium_map)
            
    return folium_map

In [6]:
map_styles = ['OpenStreetMap', 'CartoDB dark_matter', 'Mapbox Control Room', 'Stamen Watercolor']
folium_map = folium.Map(location=[48.5734, 7.7521113],
                        zoom_start=13,
                        tiles=map_styles[1],
                        width = '100%')

ministers = set(bat_data.Ministere)

min_colors = {}
for m in ministers:
    r = lambda: random.randint(0,255)
    col= '#%02X%02X%02X' % (r(),r(),r())
    min_colors[m] = col

In [84]:
# minister urls
min_url = {'Justice': 'http://justice.gouv.fr/'}

In [72]:
minister_colors

{'Economie et finances',
 'Interieur',
 'Justice',
 'Multi-occupation',
 'Services du premier ministre'}

In [78]:
bat_data.head(5)

,ID,Type,Fonction,Designation,Latitude,Longitude,Adresse,CodePostal,Ville,Ministere,Programmes,Libelle_Programme,Texte_Presentation,Depenses_T2_2017,Depenses_T3_2017,Depenses_T5_2017,Depenses_HT2_Personnel_2017,Effectif_ETPT_2017,Actif_Immobilier,Actif_Vehicules
0,IB00100028657,Bureau,Tribunal,TRIBUNAL DE GRANDE INSTANCE DE STRASBOURG,48.587333,7.747495,QU FINKMATT,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
1,IB00100036271,Bureau,Tribunal,TRIBUNAL D INSTANCE DE STRASBOURG,48.589145,7.746712,45 R DU FOSSE DES TREIZE,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
2,IB00100028684,Bureau,Tribunal,CONSEIL DE PRUD'HOMMES DE STRASBOURG,48.589834,7.755428,19 AV DE LA PAIX,67000,Strasbourg,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
3,IB00100035510,Bureau,Tribunal,TRIBUNAL ADMINISTRATIF STRASBOURG,48.591572,7.756567,31 AV DE LA PAIX,67000,Strasbourg,Services du premier ministre,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0
4,IB00100028712,Bureau,Tribunal,TRIBUNAL D INSTANCE DE MOLSHEIM,48.541025,7.494306,3 R DU MARECHAL KELLERMANN,67120,Molsheim,Justice,166,Justice judiciaire,Le programme budgétaire 166 « justice judiciai...,2.272061e+09,892981628.0,124801170.0,1.019850e+09,32488.0,3.445088e+09,2410917.0


In [79]:
for idx, row in bat_data.iterrows():        
    # generate the popup message that is shown on click.
    #print(row)
    num_layus = "Pour répondre à ses missions, en 2017: \
    <ul> <li>Il a employé {etp} agents pour une dépense totale de {dep_perso}M€</li> \
    <li>Ses dépenses de fonctionnement ont représenté {dep_fonct}M€</li> \
    <li>Il a investi {invest}M€ en 2017</li> </ul>  \
    Il dispose aussi d’un patrimoine essentiellement composé d immeubles pour une valeur de {patrimoine}M€ \
    et véhicules pour une valeur de {vehicule}M€."
    
    popup_text = "<center><strong>{} </strong></center><br> <b>Ministere:</b> {}<br> \
    <b>Fonction:</b> {}<br> <br><b>Pogramme budgétaire:</b> {} <br><br> <b>Données budgétaires et patrimoniales:</b> <br>{}"
    
    popup_text = popup_text.format(re.sub("'", ' ', row['Designation']).lower(),
                          row["Ministere"],
                          row["Fonction"],
                          #row['Effectif_ETPT_2017'],
                          row['Texte_Presentation'],
                          num_layus.format(etp = int(row['Effectif_ETPT_2017']), dep_perso = int(row['Depenses_T2_2017']*1e-6), dep_fonct = int(row['Depenses_T3_2017']*1e-6), 
                                           invest = int(row['Depenses_T5_2017']*1e-6), patrimoine = int(row['Actif_Immobilier']*1e-6), vehicule = int(row['Actif_Vehicules']*1e-6)))
    
    # radius of circles
    radius = 10

    # choose the color of the marker
    if 20>0:
        # color="#FFCE00" # orange
        # color="#007849" # green
        color="#E37222" # tangerine
    else:
        # color="#0375B4" # blue
        # color="#FFCE00" # yellow            
        color="#0A8A9F" # teal

    # add marker to the map
    folium.CircleMarker(location=(row["Latitude"],
                                  row["Longitude"]),
                        radius=radius,
                        color=color,
                        popup=popup_text,
                        fill=True).add_to(folium_map)

In [80]:
folium_map

In [47]:
folium_map.save('my_map.html')